In [8]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

False

In [6]:
class CourseRecommender:
    """
    Recommends relevant courses for missing skills using LLM.
    """
    
    def __init__(self, api_key=None):
        self.api_key = api_key or os.getenv('OPENAI_API_KEY')
        if not self.api_key:
            raise ValueError("OpenAI API key not provided. Set OPENAI_API_KEY environment variable.")
        
        self.client = OpenAI(api_key=self.api_key)
    
    def recommend_courses(self, missing_skills, num_courses=3):
        if not missing_skills:
            return {"message": "No missing skills provided"}
        
        prompt = self._create_prompt(missing_skills, num_courses)
        
        try:
            response = self.client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are a career advisor specializing in technical skill development and online learning platforms."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.7,
                max_tokens=2000
            )
            
            recommendations = self._parse_response(response.choices[0].message.content)
            return recommendations
            
        except Exception as e:
            return {"error": f"Failed to generate recommendations: {str(e)}"}
    
    def _create_prompt(self, missing_skills, num_courses):
        skills_str = ", ".join(missing_skills)
        
        prompt = f"""
I need course recommendations for the following skills: {skills_str}

For each skill, recommend {num_courses} high-quality online courses that would help someone learn this skill effectively. 

For each course recommendation, provide:
1. Course Title
2. Platform (e.g., Coursera, Udemy, edX, Udacity, YouTube, freeCodeCamp, etc.)
3. Duration (estimated time to complete)
4. Difficulty Level (Beginner, Intermediate, Advanced)
5. Brief Description (2-3 sentences about what the course covers)
6. Why it's recommended (1-2 sentences)

Format your response as a JSON object with this structure:
{{
    "skill_name": [
        {{
            "title": "Course Title",
            "platform": "Platform Name",
            "duration": "X weeks/hours",
            "difficulty": "Beginner/Intermediate/Advanced",
            "description": "Course description",
            "why_recommended": "Reason for recommendation",
            "cost": "Free/Paid/Freemium"
        }}
    ]
}}

Only return valid JSON, no additional text.
"""
        return prompt
    
    def _parse_response(self, response_text):
        try:
            start_idx = response_text.find('{')
            end_idx = response_text.rfind('}') + 1
            
            if start_idx != -1 and end_idx != 0:
                json_str = response_text[start_idx:end_idx]
                recommendations = json.loads(json_str)
                return recommendations
            else:
                return {"error": "Could not parse JSON from response", "raw_response": response_text}
                
        except json.JSONDecodeError as e:
            return {"error": f"JSON parsing error: {str(e)}", "raw_response": response_text}
    
    def save_recommendations(self, recommendations, filename="course_recommendations.json"):
        try:
            with open(filename, 'w') as f:
                json.dump(recommendations, f, indent=4)
            print(f"Recommendations saved to {filename}")
        except Exception as e:
            print(f"Error saving recommendations: {str(e)}")

In [7]:
# Example usage - replace with your actual missing skills from gap analysis
missing_skills = ["Machine Learning", "React.js", "Docker"]

recommender = CourseRecommender()
print("Generating course recommendations...")
recommendations = recommender.recommend_courses(missing_skills, num_courses=2)

# Display results
print("\n" + "="*50)
print("COURSE RECOMMENDATIONS")
print("="*50 + "\n")

if "error" in recommendations:
    print(f"Error: {recommendations['error']}")
else:
    for skill, courses in recommendations.items():
        print(f"\n📚 {skill.upper()}")
        print("-" * 40)
        for i, course in enumerate(courses, 1):
            print(f"\n{i}. {course.get('title', 'N/A')}")
            print(f"   Platform: {course.get('platform', 'N/A')}")
            print(f"   Duration: {course.get('duration', 'N/A')}")
            print(f"   Difficulty: {course.get('difficulty', 'N/A')}")
            print(f"   Cost: {course.get('cost', 'N/A')}")
            print(f"   Description: {course.get('description', 'N/A')}")

# Save to file
recommender.save_recommendations(recommendations)

Generating course recommendations...

COURSE RECOMMENDATIONS


📚 MACHINE_LEARNING
----------------------------------------

1. Machine Learning
   Platform: Coursera
   Duration: 11 weeks
   Difficulty: Intermediate
   Cost: Freemium
   Description: This course by Stanford University provides a broad introduction to machine learning, datamining, and statistical pattern recognition.

2. Machine Learning A-Z: Hands-On Python & R In Data Science
   Platform: Udemy
   Duration: 44 hours
   Difficulty: Beginner
   Cost: Paid
   Description: This course will teach you to create Machine Learning algorithms in Python and R from two Data Science experts.

📚 REACT.JS
----------------------------------------

1. React - The Complete Guide (incl Hooks, React Router, Redux)
   Platform: Udemy
   Duration: 40 hours
   Difficulty: Intermediate
   Cost: Paid
   Description: Dive in and learn React.js from scratch! Learn Reactjs, Hooks, Redux, React Routing, Animations, Next.js and way more!

2. Full-S